# Visualizing the Johns Hopkins COVID-19 time series data

**This is a work in progress.** It doesn't work yet in [Binder](https://mybinder.org/v2/gh/dotnet/interactive/master?urlpath=lab) because it relies on HTTP communication between the kernel and the Jupyter frontend.

Also, due to travel restrictions, you should run this at home on isolated compute.

*And don't forget to wash your hands.*

Since Johns Hopkins has put COVID-19 time series data on [GitHub](https://github.com/CSSEGISandData/COVID-19), let's take a look at it. We can download it using PowerShell:

In [1]:
#!pwsh
Invoke-WebRequest -Uri "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv" -OutFile "./Confirmed.csv"
Invoke-WebRequest -Uri "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv" -OutFile "./Deaths.csv"
Invoke-WebRequest -Uri "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv" -OutFile "./Recovered.csv"

It needs a little cleaning up:

In [2]:
using System.IO;
using System.Text.RegularExpressions;

Clean("Confirmed.csv");
Clean("Deaths.csv");
Clean("Recovered.csv");

void Clean(string filePath)
{
    var raw = File.ReadAllText(filePath);
    var regex = new Regex("\\\"(.*?)\\\"");
    var cleaned = regex.Replace(raw, m => m.Value.Replace(",", " in "));  
    File.WriteAllText(filePath, cleaned);
}

Next, let's load it into a data frame.

In [3]:
#r "nuget:Microsoft.Data.Analysis,0.2.0"

Installed package Microsoft.Data.Analysis version 0.2.0

In [4]:
using Microsoft.Data.Analysis;

var deaths = DataFrame.LoadCsv("./Deaths.csv");
var confirmed = DataFrame.LoadCsv("./Confirmed.csv");
var recovered = DataFrame.LoadCsv("./Recovered.csv");

var offset = 4;

var date = deaths.Columns[offset].Name;
var deathFiltered = deaths[deaths.Columns[offset].ElementwiseNotEquals(0)];
var confirmedFiltered = confirmed[confirmed.Columns[offset].ElementwiseNotEquals(0)];
var recoveredFiltered = recovered[recovered.Columns[offset].ElementwiseNotEquals(0)];

var current = confirmed.Columns[offset] - (deaths.Columns[offset] + recovered.Columns[offset]);
var currentLocFiltered = confirmed[current.ElementwiseNotEquals(0)];
var currentDataFiltered = current.Clone(current.ElementwiseNotEquals(0));
                                                                                   
var deathsSeries = new {
    latitude = deathFiltered["Lat"],
    longitude = deathFiltered["Long"],
    data = deathFiltered.Columns[offset]
};
var confirmedSeries = new {
    latitude = confirmedFiltered["Lat"],
    longitude = confirmedFiltered["Long"],
    data = confirmedFiltered.Columns[offset]
};
var recoveredSeries = new {
    latitude = recoveredFiltered["Lat"],
    longitude = recoveredFiltered["Long"],
    data = recoveredFiltered.Columns[offset]
};
                                                                                   
var currentSeries = new {
    latitude = currentLocFiltered["Lat"],
    longitude = currentLocFiltered["Long"],
    data = currentDataFiltered
};

"Ready."

Ready.

Because we've stored our data in top-level variables (`deathsSeries`, `confirmedSeries`, `recoveredSeries`, etc.) in the C# kernel, they're accessible from JavaScript by calling `interactive.csharp.getVariable`. The data will be returned as JSON and we can plot it using the library of our choice, pulled in using [RequireJS](https://requirejs.org/). 

We'll use [Plotly](https://plot.ly/).

In [8]:
#!javascript
var plot = function() {

    plotlyJs_covid_require = require.config({
        context: "COVID",
        paths: {
            plotly: "https://cdn.plot.ly/plotly-latest.min"
        }
    });
    plotlyJs_covid_require(["plotly"], (Plotly) => {
        function updateCovidPlot(){
             Promise.all([
                 interactive.csharp.getVariable("deathsSeries"), 
                 interactive.csharp.getVariable("confirmedSeries"), 
                 interactive.csharp.getVariable("recoveredSeries"),
                 interactive.csharp.getVariable("currentSeries"),
                 interactive.csharp.getVariable("date")])
            .then(data => {
                Plotly.animate("plotlyChartCovid", {
                    data:[
                        {
                            lat: data[2].latitude,
                            lon: data[2].longitude,
                            text: data[2].data
                        },
                        {
                            lat: data[0].latitude,
                            lon: data[0].longitude,
                            text: data[0].data
                        },
                        {
                            lat: data[1].latitude,
                            lon: data[1].longitude,
                            text: data[1].data
                        },
                        {
                            lat: data[3].latitude,
                            lon: data[3].longitude,
                            text: data[3].data
                        }],
                    layout:{
                        title: "COVID-19 "+ data[4]
                    }
                });
            });
        }
            Promise.all([
                interactive.csharp.getVariable("deathsSeries"), 
                interactive.csharp.getVariable("confirmedSeries"), 
                interactive.csharp.getVariable("recoveredSeries"),
                interactive.csharp.getVariable("currentSeries"),
                interactive.csharp.getVariable("date")])
            .then(data => {
                var recovered = {
                    name: "recovered",
                    type: "scattergeo",
                    mode: "markers",
                    geo: "geo1",
                    lat: data[2].latitude,
                    lon: data[2].longitude,
                    text: data[2].data,
                    marker: {
                        symbol: "square",
                        colorscale: "Viridis",
                    }
                };

                var deaths = {
                    name: "death",
                    type: "scattergeo",
                    geo: "geo2",
                    mode: "markers",
                    lat: data[0].latitude,
                    lon: data[0].longitude,
                    text: data[0].data,
                    marker: {
                        symbol: "circle",
                        colorscale: "Viridis",
                    }
                };

                var confirmed = {
                    name: "confirmed",
                    type: "scattergeo",
                    geo: "geo3",
                    mode: "markers",
                    lat: data[1].latitude,
                    lon: data[1].longitude,
                    text: data[1].data,
                    marker: {
                        symbol: "diamond",
                        colorscale: "Viridis",
                    }
                };
                
                var current = {
                    name: "current",
                    type: "scattergeo",
                    geo: "geo4",
                    mode: "markers",
                    lat: data[3].latitude,
                    lon: data[3].longitude,
                    text: data[3].data,
                    marker: {
                        symbol: "triangle",
                        colorscale: "Viridis",
                    }
                };

                var traces = [recovered, deaths, confirmed, current];

                var layout = {
                    title: "COVID-19 "+ data[4],
                    grid: { columns: 4, rows: 1 },
                    geo1: {
                        scope: "world",
                        showland: true,
                        landcolor: "rgb(250,250,250)",
                        domain: {
                            row: 0,
                            column: 0
                        }
                    },
                    geo2: {
                        scope: "world",
                        showland: true,
                        landcolor: "rgb(250,250,250)",
                        domain: {
                            row: 0,
                            column: 1
                        }
                    },
                    geo3: {
                        scope: "world",
                        showland: true,
                        landcolor: "rgb(250,250,250)",
                        domain: {
                            row: 0,
                            column: 2
                        }
                    },
                    geo4: {
                        scope: "world",
                        showland: true,
                        landcolor: "rgb(250,250,250)",
                        domain: {
                            row: 0,
                            column: 3
                        }
                    }
                };

                Plotly.newPlot("plotlyChartCovid", traces, layout);
                setInterval(() => updateCovidPlot(), 100);
            });
    });
};

Notice the `setInterval` call near the end of the previous cell. This rechecks the data in the kernel and updates the plot.

Back on the kernel, we can now update the data so that the kernel can see it.

Yes, this is a contrived example, and we're planning to support true streaming data, but it's a start.

In [11]:
#!html
<div id="plotlyChartCovid"></div>

#!js
plot();

#!csharp
for(var i = offset; i <  deaths.Columns.Count; i++){
    await Task.Delay(100);
    date = deaths.Columns[i].Name;
    deathFiltered = deaths[deaths.Columns[i].ElementwiseNotEquals(0)];
    confirmedFiltered = confirmed[confirmed.Columns[i].ElementwiseNotEquals(0)];
    recoveredFiltered = recovered[recovered.Columns[i].ElementwiseNotEquals(0)];
    
    current = confirmed.Columns[i] - (deaths.Columns[i] + recovered.Columns[i]);
    currentLocFiltered = confirmed[current.ElementwiseNotEquals(0)];
    currentDataFiltered = current.Clone(current.ElementwiseNotEquals(0));

    deathsSeries = new {
        latitude = deathFiltered["Lat"],
        longitude = deathFiltered["Long"],
        data = deathFiltered.Columns[i]
    };
    confirmedSeries = new {
        latitude = confirmedFiltered["Lat"],
        longitude = confirmedFiltered["Long"],
        data = confirmedFiltered.Columns[i]
    };
    recoveredSeries = new {
        latitude = recoveredFiltered["Lat"],
        longitude = recoveredFiltered["Long"],
        data = recoveredFiltered.Columns[i]
    };
    currentSeries = new {
        latitude = currentLocFiltered["Lat"],
        longitude = currentLocFiltered["Long"],
        data = currentDataFiltered
    };
}